pandas                        1.4.2
numpy                         1.21.5
matplotlib                    3.5.1
matplotlib-inline             0.1.2
scipy                         1.7.3
seaborn                       0.11.2
statsmodels                   0.13.2

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
data = pd.read_csv('cluster_df.csv')

In [6]:
cluster_df=data.copy()

In [7]:
data.drop(['Unnamed: 0'],axis=1,inplace=True)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34084 entries, 0 to 34083
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   birth_year            34084 non-null  int64  
 1   credit_score          34084 non-null  float64
 2   yearly_income         34084 non-null  float64
 3   income_type           34084 non-null  int64  
 4   employment_type       34084 non-null  int64  
 5   houseown_type         34084 non-null  int64  
 6   desired_amount        34084 non-null  float64
 7   purpose               34084 non-null  int64  
 8   existing_loan_cnt     34084 non-null  float64
 9   existing_loan_amt     34084 non-null  float64
 10  gender                34084 non-null  float64
 11  time                  34084 non-null  float64
 12  cluster               34084 non-null  int64  
 13  count                 34084 non-null  int64  
 14  kmeans_minmax_label3  34084 non-null  int64  
dtypes: float64(7), int6

In [9]:
data1=data.copy()
data1[['income_type','employment_type','houseown_type','purpose']]=data[['income_type','employment_type','houseown_type','purpose']].astype('category')
data1[['birth_year','gender','kmeans_minmax_label3']]=data[['birth_year','gender','kmeans_minmax_label3']].astype('category')

df=data1.copy()
df['time']=np.sqrt(np.sqrt(data1['time']))
df['desired_amount']=np.sqrt(data1['desired_amount'])
df['yearly_income']=np.sqrt(np.sqrt(data1['yearly_income']))
df['existing_loan_amt']=np.sqrt(data1['existing_loan_amt'])
df['existing_loan_cnt']=np.sqrt(np.sqrt(np.sqrt(data1['existing_loan_cnt'])))

from sklearn.preprocessing import MinMaxScaler
X_features_scaled = MinMaxScaler().fit_transform(df)
data_scaled=pd.DataFrame(X_features_scaled, columns=data.columns)

In [10]:
X_features_scaled.shape

(34084, 15)

# 분산분석 : 여러 집단 간에 통계적으로 차이가 있는지
확인해볼 값 
* 신용점수는 같은 등급대라 비교할 필요 없을듯
* 연소득이 각 그룹별로 차이가 있는지
* 희망금액이 각 그룹별로 차이가 있는지
* 기대출금액이 차이가 있는지


정규성을 따른다는 의미는 통계량을 재해석하면 왜도와 첨도가 얼마나 0에 가깝냐라는 의미이다.
정규분포의 왜도, 첨도가 0이기 때문에 이로부터 많이 벗어났는지에 대한 기준으로 표준오차를 사욜한다.
근데 왜도, 첨도의 표준오차는 데이터 수에 의존적이기 때문에 수가 많아진다면 표준오차는 작아진다.
표준오차가 작아지면, 동일한 왜도나 첨도 값일지라도 통계적으로 유의미하게 만들며 이는 데이터가 정규성이 아니다라는 주장을 지지하게 만든다.
통계해례 98p, 203p

* Pr(>F)가 p-value. 이 값이 유의수준 0.05하에서 귀무가설을 기각함. 따라서 세가지 종에따른 꽃받침 폭이 모두 동일하지는 않다고 결론내릴 수 있다. 즉, 종별 꽃받침 폭의 평균값들 중에서 적어도 어느 하나의 종은 통계적으로 유의한 차이가 있다.

* 분산분석가정 
1. 정규성 : 각 그룹의 데이터는 정규분포 형태를 가진다. 
정의 : 모든 데이터는 정규분포를 따르는 모집단들로부터 추출됨
* shapiro : 데이터 수가 5000개 이상일 때에는 p값의 의미가 없어진다. 군집분석하기 전에 왜도가 0에 근사하게 데이터를 지수변환, 로그변환 했기에 모집단은 정규분포를 따른다고 가정할 수 있다.


2. 등분산 : 각 그룹의 분산은 서로 같은 분산이다. 
* 예외 ) 데이터의 수가 아주 많은 경우에는, 첫 번째 조건인 정규성을 가지지 않아도 된다. 만약 그룹간의 데이터 수가 비슷하고 어느 두 그룹 간의 분산의 비 ratio of any two variances 가 모두 4를 넘지 않는 경우에는, 두 번째 조건인 등분산을 가지지 않아도 된다.

3. 독립성 : 각 그룹에 대해 관찰은 서로 독립적이며, 그룹간 서로 영향을 주지 않아야한다.



** 중심극한정리 : 모집단 분포에 상관없이 큰 표본들의 표본평균 분포가 정규분포에 수렴한다는 점을 이용하여 z값을 구해 확률값을 구할 수 있고, 수학적 확률 추정이 가능하다.

### 군집별로 희망금액은 차이가 있다.
반올림 해서 57045288, 81229684, 49544603

In [11]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
model = ols('desired_amount~ C(kmeans_minmax_label3)', cluster_df).fit()
anova_lm(model)

#귀무가설 : 집단별로 희망금액 평균은 모두 같다.
#대립가설 : 적어도 하나의 집단에 대한 희망금액의 평균값에는 차이가 있다.
# Pr(>F)가 p-value. 이 값이 유의수준 0.05하에서 귀무가설을 기각한다. 따라서 세 군집의 희망금액 평균이 모두 동일하지는 않다고 결론내릴 수 있다.

,df,sum_sq,mean_sq,F,PR(>F)
C(kmeans_minmax_label3),2.0,1.891264e+10,9.456320e+09,347.263753,5.028187e-150
Residual,34081.0,9.280578e+11,2.723094e+07,NaN,NaN


In [12]:
model = ols('yearly_income~ C(kmeans_minmax_label3)', cluster_df).fit()
anova_lm(model)

#귀무가설 : 집단별로 연소득 평균은 모두 같다.
#대립가설 : 적어도 하나의 집단에 대한 연소득 평균값에는 차이가 있다.
# Pr(>F)가 p-value. 이 값이 유의수준 0.05하에서 귀무가설을 기각한다. 따라서 세 군집의 연소득 평균이 모두 동일하지는 않다고 결론내릴 수 있다.


,df,sum_sq,mean_sq,F,PR(>F)
C(kmeans_minmax_label3),2.0,1.142950e+06,571475.069482,1410.246053,0.0
Residual,34081.0,1.381067e+07,405.230753,NaN,NaN


In [13]:
# 이원배치 
##이건 cluster별로 해야겠는데 , 집단별로 표본 수가 동일할 때
##예를 들면 기대출금액이 목적과 성별에 따라 평균 차이가 난다고 볼 수 있나.
## 세번째 자료에서 상호작용 효과를 알 수 있음 


from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
model = ols('existing_loan_amt ~ C(purpose) * C(gender)', cluster_df).fit()
anova_lm(model)


,df,sum_sq,mean_sq,F,PR(>F)
C(purpose),7.0,1.950619e+10,2.786598e+09,155.631150,2.668618e-227
C(gender),1.0,7.192464e+09,7.192464e+09,401.698198,7.656834e-89
C(purpose):C(gender),7.0,9.662879e+08,1.380411e+08,7.709580,2.429939e-09
Residual,34068.0,6.099924e+11,1.790514e+07,NaN,NaN


In [15]:
!pip install pingouin

  Created wheel for pingouin: filename=pingouin-0.5.2-py3-none-any.whl size=196207 sha256=5aa73faf05b9edd632e10773629750f43ce269505b238b5d4b8048ad122ae3da
  Stored in directory: c:\users\jus68\appdata\local\pip\cache\wheels\c0\9f\92\4c574395e1e8e5e08cf73dcb76815a7eaa62921833b6b0f6ad
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7048 sha256=ee08dafd576876e59497ddce59b874bf0d9ad9bfe5e36dbf5ba295b6a9b56e07
  Stored in directory: c:\users\jus68\appdata\local\pip\cache\wheels\04\bb\0d\2d02ec45f29c48d6192476bfb59c5a0e64b605e7212374dd15
Successfully built pingouin littleutils
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.1.2
    Uninstalling scikit-learn-1.1.2:
      Successfully uninstalled scikit-learn-1.1.2


ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\jus68\\anaconda3\\Lib\\site-packages\\~klearn\\.libs\\vcomp140.dll'
Consider using the `--user` option or check the permissions.



In [16]:
# 등분산성을 만족하지 못할 때
import pingouin as pg
pg.welch_anova(dv = 'yearly_income', between = 'income_type', data = cluster_df)

# income type에 따라 연소득의 차이가 있다. 

ModuleNotFoundError: No module named 'pingouin'

In [ ]:
pg.welch_anova(dv = 'employment_type', between = 'income_type', data = cluster_df)

# 고용형태에 따라 연소득의 차이가 있다. 

In [ ]:
## 이원배치 분산분석

cluster_df.boxplot(column='yearly_income', by='kmeans_minmax_label3')

In [ ]:
# 이원배치 
##집단별로 표본 수가 동일할 때 기대출금액이 목적과 성별에 따라 평균 차이가 난다고 볼 수 있다.
# 둘의 상호작용 효과 또한 나타난 것을 알 수 있다.
## 세번째 자료에서 상호작용 효과를 알 수 있음 


from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
model = ols('existing_loan_amt ~ C(purpose) * C(gender)', cluster_df).fit()
anova_lm(model)


 일원분산분석과 비교해서 상호작용효과를 검증/분석하는게 중요
상호작용효과가 유의미한 경우, 단순효과분석 결과를 상세하게 첨부

* 기대출금액에 대하여 성별과 목적를 요인으로 하는 이원분산분석을 실시한 결과, 목적 주효과는 유의미하였으며(F =155.63115, p < 0.05), 성별의 주효과 또한 유의미하고(F=401.69820, p < 0.05), 두 변수의 유의미한 상호작용효과 또한 발견할 수 있었다(F= 	138041127.25421, p < 0.05).

In [ ]:
# 이원분산분석
# 유의하지 않다.

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
model = ols('existing_loan_amt ~ C(kmeans_minmax_label3) * C(gender)', cluster_df).fit()
anova_lm(model,typ=3)

# 다 유의하지 않다.

In [ ]:
## 두 변수가 독립적인지 - 카이제곱 검증
# 귀무 : 두 변수는 독립이다. 대립 : 두 변수는 독립아니다.
import scipy.stats
# 검정통계량, p값, 자유도
chi2, pvalue, dof, expected = scipy.stats.chi2_contingency(cluster_df[['desired_amount','credit_score']])
print(chi2, pvalue, dof)
